**O objetivo do exercício desta semana é construir um sistema de RAG combinando busca e LLMs.**

**Etapa 1: Segmentação do texto** - o método usado para segmentação (aka. chucking e janelamento) é livre. A sugestão é usar as ferramentas do langchain.

**Etapa 2: Computação das embeddings e indexação** - A ideia é usar busca densa, usando um modelo de embeddings para criar representações vetoriais para os segmentos obtidos na etap 1. Fiquem livres para usar qualquer modelo, mas sugerimos os modelos do sentence-transformers. Usem o FAISS para criar e gerenciar o índice.

**Etapa 3: Geração das respostas (prompt)** - o LLM usado também é livre. Não fica muito caro usar o gpt-4o-mini, opções free são a API da groq e Ollama. Para esta etapa, usem apenas 150 perguntas (10% do dataset).

**Etapa 4: Avaliação**  -  É talvez o ponto mais importante do trabalho. Sugerimos uma avaliação com a métrica F1-bag-of-words (ver slide 167).



In [1]:
# Instala as bibliotecas langchain, sentence_transformers e groq
!pip install -qU langchain sentence_transformers groq

# Instala bibliotecas complementares do langchain, como integração com Hugging Face e a comunidade langchain
!pip install -qU langchain-huggingface langchain-community

# Instala a biblioteca FAISS para operações eficientes de pesquisa de vetores em CPU
!pip install -qU faiss-cpu

# Instala o módulo langchain-text-splitter, usado para dividir textos em partes menores
!pip install -qU langchain-text-splitter

# Baixa o conjunto de dados de teste IIRC
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json

# Baixa o arquivo compactado contendo artigos de contexto
!wget https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz

# Extrai os arquivos do contexto dos artigos compactados
!tar -xf context_articles.tar.gz

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
--2024-10-02 16:39:08--  https://iirc-dataset.s3.us-west-2.amazonaws.com/iirc_test.json
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 3.5.82.196, 52.92.147.250, 3.5.86.133, ...
Connecting to iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)|3.5.82.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874825 (2.7M) [application/json]
Saving to: ‘iirc_test.json.2’

iirc_test.json.2    100%[===================>]   2.74M  4.60MB/s    in 0.6s    

2024-10-02 16:39:10 (4.60 MB/s) - ‘iirc_test.json.2’ saved [2874825/2874825]

--2024-10-02 16:39:10--  https://iirc-dataset.s3.us-west-2.amazonaws.com/context_articles.tar.gz
Resolving iirc-dataset.s3.us-west-2.amazonaws.com (iirc-dataset.s3.us-west-2.amazonaws.com)... 3.

In [2]:
# Importa o divisor de texto de caracteres recursivo da biblioteca langchain_text_splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Importa a classe para embeddings da Hugging Face da biblioteca langchain_huggingface
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

# Importa métricas de avaliação do scikit-learn: precisão, recall e F1-score
from sklearn.metrics import precision_score, recall_score, f1_score

# Importa Counter para contagem de elementos
from collections import Counter

# Importa a biblioteca Groq
from groq import Groq

# Importa a barra de progresso do tqdm
from tqdm import tqdm

# Importa a biblioteca NumPy para operações matemáticas e manipulação de arrays
import numpy as np

# Importa o módulo JSON para trabalhar com arquivos JSON
import json

# Importa a biblioteca FAISS para realizar operações de busca em vetores
import faiss


from concurrent.futures import ThreadPoolExecutor, as_completed
from typing import List, Union, Dict, Any


In [3]:
# Define a chave da API diretamente no código
API_KEY = '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'

In [4]:
# @title Carregar dados

# Carrega os artigos de contexto a partir do arquivo JSON "context_articles.json"
context_articles = json.load(open("context_articles.json", 'r'))

# Carrega o conjunto de dados de teste a partir do arquivo JSON "iirc_test.json"
data = json.load(open("iirc_test.json", 'r'))


In [5]:
# Função para excluir respostas do tipo 'none' e adicionar o título às perguntas
def exclude_none_answers(data):
    filtered_data = []
    for item in data:
        # Filtra as perguntas válidas e adiciona o título a cada uma sem modificar o original
        filtered_questions = [
            {**question, 'title': item['title']}
            for question in item['questions']
            if question['answer']['type'] != 'none'
        ]
        # Se houver perguntas filtradas, adiciona ao resultado
        if filtered_questions:
            filtered_item = item.copy()
            filtered_item['questions'] = filtered_questions
            filtered_data.append(filtered_item)
    return filtered_data

# Imprime o número total de itens antes da filtragem
print(f'Total de itens: {len(data)}')

# Filtra as respostas do conjunto de dados original
filter_data = exclude_none_answers(data)

# Imprime o número de itens após a filtragem
print(f'Itens após filtragem: {len(filter_data)}')

# Exibe uma amostra dos dados filtrados
print("\n===> Mostrar um exemplo <===")
print(filter_data[0])


Total de itens: 514
Itens após filtragem: 435

===> Mostrar um exemplo <===
{'questions': [{'answer': {'type': 'span', 'answer_spans': [{'text': 'sky and thunder god', 'passage': 'zeus', 'type': 'answer', 'start': 83, 'end': 102}]}, 'question': 'What is Zeus know for in Greek mythology?', 'context': [{'text': 'he Palici the sons of Zeus', 'passage': 'main', 'indices': [684, 710]}, {'text': 'in Greek mythology', 'passage': 'main', 'indices': [137, 155]}, {'text': 'Zeus (British English , North American English ; , Zeús ) is the sky and thunder god in ancient Greek religion', 'passage': 'Zeus', 'indices': [0, 110]}], 'question_links': ['Greek mythology', 'Zeus'], 'title': 'Palici'}], 'text': "The Palici (Παλικοί in Greek), or Palaci, were a pair of indigenous Sicilian chthonic deities in Roman mythology, and to a lesser extent in Greek mythology. They are mentioned in Ovid's Metamorphoses V, 406, and in Virgil's Aeneid IX, 585. Their cult centered on three small lakes that emitted sulphu

In [6]:
# @title Carregar documentos
# Adaptado de Visconde https://github.com/neuralmind-ai/visconde

# Número de perguntas que serão usadas para carregar documentos
num_questions = 150

documents = []  # Lista para armazenar os documentos carregados
all_titles = set()  # Conjunto para armazenar os títulos de documentos já adicionados

# Itera sobre os primeiros 'num_questions' itens dos dados filtrados
for item in filter_data[:num_questions]:
    item_title = item['title']
    item_title_lower = item_title.lower()
    # Verifica se o título do item ainda não foi adicionado
    if item_title_lower not in all_titles:
        # Adiciona o documento contendo o título e o texto do item
        documents.append({
            "title": item_title,
            "content": item["text"]
        })
        all_titles.add(item_title_lower)  # Adiciona o título ao conjunto de títulos utilizados

    # Itera sobre os links do item atual, se existirem
    for link in item.get("links", []):
        link_target = link['target']
        link_target_lower = link_target.lower()
        # Verifica se o alvo do link está nos artigos de contexto e ainda não foi adicionado
        if link_target_lower in context_articles and link_target_lower not in all_titles:
            # Adiciona o documento do artigo de contexto correspondente
            documents.append({
                "title": link_target,
                "content": context_articles[link_target_lower]
            })
            all_titles.add(link_target_lower)  # Adiciona o título ao conjunto de títulos utilizados
        else:
            # Imprime uma mensagem informando que o documento já foi carregado ou não está disponível
            print(f"Documento já carregado ou não encontrado: {link_target_lower}")

# Imprime o número total de documentos carregados
print(f'Total de documentos: {len(documents)}')


Documento já carregado ou não encontrado: 9th paratroopers assault regiment "col moschin"
Documento já carregado ou não encontrado: goldfinger (film)
Documento já carregado ou não encontrado: list of international cricket council members
Documento já carregado ou não encontrado: icc americas championship
Documento já carregado ou não encontrado: the rev
Documento já carregado ou não encontrado: avenged sevenfold
Documento já carregado ou não encontrado: fox footy
Documento já carregado ou não encontrado: herald sun
Documento já carregado ou não encontrado: fox footy
Documento já carregado ou não encontrado: herald sun
Documento já carregado ou não encontrado: united states
Documento já carregado ou não encontrado: judeo-iraqi arabic
Documento já carregado ou não encontrado: maya civilization
Documento já carregado ou não encontrado: black watch
Documento já carregado ou não encontrado: suicidal tendencies
Documento já carregado ou não encontrado: western hockey league
Documento já carr

In [7]:
#@title Divisão de texto em pedaços (Chunking)

# Cria uma instância do divisor de texto recursivo com uma lista de separadores padrão
text_splitter = RecursiveCharacterTextSplitter(
    # Lista de separadores padrão
    separators=["\n\n", "\n", " ", ""],
    chunk_size=1500,        # Tamanho máximo do pedaço (chunk) em caracteres
    chunk_overlap=100,      # Sobreposição entre pedaços para manter contexto
    length_function=len,    # Função para calcular o comprimento de cada pedaço
    is_separator_regex=False,  # Define se os separadores são regex ou não (nesse caso, não são)
)

# Cria uma lista com os conteúdos dos documentos
contents = [doc['content'] for doc in documents]

# Função para dividir um único conteúdo em pedaços
def split_content(content):
    return text_splitter.split_text(content)

# Divisão do conteúdo em pedaços menores (chunking) usando processamento paralelo
def split_contents_in_chunks(contents):
    chunks = []
    with ThreadPoolExecutor() as executor:
        futures = {executor.submit(split_content, content): content for content in contents}
        for future in as_completed(futures):
            try:
                chunks.extend(future.result())
            except Exception as e:
                print(f"Erro ao processar conteúdo: {e}")
    return chunks

# Chama a função para dividir os conteúdos
chunks = split_contents_in_chunks(contents)

# Imprime o número total de pedaços gerados
print(f'Total de pedaços: {len(chunks)}')

# Imprime o comprimento dos primeiros 10 pedaços
print("\nComprimento dos primeiros 10 pedaços:")
print([len(text) for text in chunks[:10]])

# Exibe um exemplo do primeiro pedaço gerado
print("\n===> Mostrar um exemplo <===")
print(chunks[0])


Total de pedaços: 59941

Comprimento dos primeiros 10 pedaços:
[1290, 1461, 1233, 1159, 281, 1118, 1493, 578, 1378, 1348]

===> Mostrar um exemplo <===
The Scottsdale Scorpions are a baseball team that plays in the East Division of the <a href="Arizona%20Fall%20League">Arizona Fall League</a> located in <a href="Scottsdale%2C%20Arizona">Scottsdale, Arizona</a>. They play their home games at <a href="Scottsdale%20Stadium">Scottsdale Stadium</a>.

Team history.In the fall of 1994, the team gained worldwide media attention, when <a href="Michael%20Jordan">Michael Jordan</a> joined the Scorpions after playing his first minor league baseball season with the Double-A <a href="Birmingham%20Barons">Birmingham Barons</a> in <a href="Birmingham%2C%20Alabama">Birmingham, Alabama</a>.

The Scorpions won their first championship in 1996 against the <a href="Mesa%20Saguaros">Mesa Saguaros</a>.

The Scorpions would make the championship game in 2002, 2004, and 2005 but would fail to win it. For the 2

In [8]:
# @title Baixar modelo para calcular embeddings

def load_embedding_model(model_name: str = "sentence-transformers/all-mpnet-base-v2") -> HuggingFaceEmbeddings:
    """
    Carrega o modelo de embeddings da Hugging Face.

    Args:
        model_name (str): Nome do modelo a ser carregado.

    Returns:
        HuggingFaceEmbeddings: Instância do modelo de embeddings.
    """
    try:
        model_emb = HuggingFaceEmbeddings(model_name=model_name)
        return model_emb
    except Exception as e:
        print(f"Erro ao carregar o modelo {model_name}: {e}")
        raise

def generate_embedding(model_emb: HuggingFaceEmbeddings, text: str) -> list:
    """
    Gera o embedding para um texto fornecido.

    Args:
        model_emb (HuggingFaceEmbeddings): Modelo de embeddings.
        text (str): Texto de entrada.

    Returns:
        list: Vetor de embedding.
    """
    try:
        query_result = model_emb.embed_query(text)
        return query_result
    except Exception as e:
        print(f"Erro ao gerar embedding para o texto: {e}")
        raise

# Cache para o modelo de embeddings
embedding_model_cache = {}

def get_embedding_model(model_name: str) -> HuggingFaceEmbeddings:
    """
    Obtém o modelo de embeddings, utilizando cache para evitar recargas desnecessárias.

    Args:
        model_name (str): Nome do modelo a ser obtido.

    Returns:
        HuggingFaceEmbeddings: Instância do modelo de embeddings.
    """
    if model_name not in embedding_model_cache:
        embedding_model_cache[model_name] = load_embedding_model(model_name)
    return embedding_model_cache[model_name]

# Nome do modelo a ser usado
model_name = "sentence-transformers/all-mpnet-base-v2"

# Obtém o modelo de embeddings
model_emb = get_embedding_model(model_name)

# Define um texto de exemplo para gerar embeddings
text = "This is a test document."

# Gera o embedding para a consulta usando o texto de exemplo
query_result = generate_embedding(model_emb, text)

# Exemplo para mostrar como gerar embeddings
def display_embedding_example(text: str, embedding: list):
    """
    Exibe um exemplo de como gerar embeddings.

    Args:
        text (str): Texto de entrada.
        embedding (list): Vetor de embedding gerado.
    """
    print("\n\nExemplo para gerar embeddings")
    print(f'Texto de entrada: {text}')
    print(f'Tamanho do embedding: {len(embedding)}')
    print(f'Exemplo do embedding: {embedding[:3]}')  # Mostra os primeiros 3 valores do embedding

# Chama a função para exibir o exemplo
display_embedding_example(text, query_result)


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavio



Exemplo para gerar embeddings
Texto de entrada: This is a test document.
Tamanho do embedding: 768
Exemplo do embedding: [-0.04895172640681267, -0.039861924946308136, -0.021562771871685982]


In [9]:
# @title Calcular embeddings para cada pedaço (chunk)

# Função para calcular o embedding de um único chunk
def compute_embedding(chunk: str, model_emb) -> List[float]:
    try:
        embedding = model_emb.embed_query(chunk)
        return embedding
    except Exception as e:
        print(f"Erro ao calcular embedding para um chunk: {e}")
        return None  # Retorna None em caso de erro

# Função para calcular embeddings para uma lista de chunks
def compute_embeddings_for_chunks(chunks: List[str], model_emb, max_workers: int = 4) -> List[List[float]]:
    embeddings_list = []  # Lista para armazenar os embeddings de cada pedaço

    # Utiliza ThreadPoolExecutor para processamento paralelo
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submete todas as tarefas ao executor
        futures = {executor.submit(compute_embedding, chunk, model_emb): chunk for chunk in chunks}

        # Utiliza tqdm para exibir a barra de progresso
        for future in tqdm(as_completed(futures), total=len(futures), desc="Calculando embeddings"):
            chunk = futures[future]
            embedding = future.result()
            if embedding is not None:
                embeddings_list.append(embedding)
            else:
                print(f"Embedding não calculado para o chunk: {chunk[:30]}...")  # Mostra início do chunk problemático

    return embeddings_list

# Chama a função para calcular os embeddings dos chunks
embeddings_list = compute_embeddings_for_chunks(chunks, model_emb)

# Imprime o número total de embeddings gerados
print(f'Total de embeddings: {len(embeddings_list)}')

# Verifica se a lista de embeddings não está vazia antes de acessar o primeiro elemento
if embeddings_list:
    # Imprime a dimensão dos embeddings gerados
    print(f"Dimensão do embedding: {len(embeddings_list[0])}")
else:
    print("Nenhum embedding foi calculado.")


Calculando embeddings: 100%|██████████| 59941/59941 [24:49<00:00, 40.25it/s]


Total de embeddings: 59941
Dimensão do embedding: 768


In [10]:
# @title Criar índice FAISS

def create_faiss_index(embeddings_list, index_file_path='index_file.index'):
    """
    Cria e salva um índice FAISS a partir de uma lista de embeddings.

    Args:
        embeddings_list (list): Lista de embeddings (listas ou arrays numpy).
        index_file_path (str): Caminho do arquivo para salvar o índice FAISS.

    Returns:
        index: Índice FAISS criado.
    """
    import numpy as np
    import faiss

    # Verifica se a lista de embeddings não está vazia
    if not embeddings_list:
        raise ValueError("A lista de embeddings está vazia.")

    # Converte a lista de embeddings para um array numpy
    embeddings_array = np.array(embeddings_list).astype('float32')

    # Verifica se todos os embeddings têm a mesma dimensão
    dimension = embeddings_array.shape[1]
    if not all(len(embedding) == dimension for embedding in embeddings_array):
        raise ValueError("Todos os embeddings devem ter a mesma dimensão.")

    # Escolha do tipo de índice FAISS com base no tamanho do dataset
    num_embeddings = embeddings_array.shape[0]
    if num_embeddings < 10000:
        # Para conjuntos pequenos, IndexFlatL2 é adequado
        index = faiss.IndexFlatL2(dimension)
    else:
        # Para conjuntos maiores, usar índice IVF (Inverted File Index)
        nlist = 100  # Número de listas (ajuste conforme necessário)
        quantizer = faiss.IndexFlatL2(dimension)
        index = faiss.IndexIVFFlat(quantizer, dimension, nlist, faiss.METRIC_L2)
        # Treina o índice IVF
        index.train(embeddings_array)

    # Adiciona os embeddings ao índice
    index.add(embeddings_array)

    # Salva o índice em um arquivo para uso posterior
    faiss.write_index(index, index_file_path)

    print(f"Índice FAISS criado e salvo em '{index_file_path}'.")

    return index

# Exemplo de uso da função
index_file_path = 'index_file.index'
index = create_faiss_index(embeddings_list, index_file_path)


Índice FAISS criado e salvo em 'index_file.index'.


In [11]:
# @title Carregar o índice FAISS salvo

def load_faiss_index(index_file_path: str = 'index_file.index'):
    """
    Carrega o índice FAISS a partir de um arquivo salvo.

    Args:
        index_file_path (str): Caminho do arquivo de índice FAISS.

    Returns:
        faiss.Index: Índice FAISS carregado.

    Raises:
        FileNotFoundError: Se o arquivo de índice não for encontrado.
        Exception: Para outros erros durante o carregamento do índice.
    """
    try:
        index = faiss.read_index(index_file_path)
        print(f"Índice FAISS carregado de '{index_file_path}'.")
        return index
    except FileNotFoundError:
        print(f"Erro: Arquivo de índice '{index_file_path}' não encontrado.")
        raise
    except Exception as e:
        print(f"Erro ao carregar o índice FAISS: {e}")
        raise

# Exemplo de uso da função
index = load_faiss_index('index_file.index')


Índice FAISS carregado de 'index_file.index'.


In [12]:
# @title Recuperar pedaços (chunks)

def generate_query_embedding(query: str, model) -> np.ndarray:
    """
    Gera o embedding para a consulta fornecida.

    Args:
        query (str): Texto da consulta.
        model: Modelo de embeddings utilizado.

    Returns:
        np.ndarray: Embedding da consulta.
    """
    try:
        query_embedding = model.embed_query(query)
        query_embedding = np.array([query_embedding]).astype('float32')
        return query_embedding
    except Exception as e:
        print(f"Erro ao gerar embedding para a consulta: {e}")
        raise

def search_index(query_embedding: np.ndarray, index, k: int) -> np.ndarray:
    """
    Realiza a busca no índice FAISS com o embedding da consulta.

    Args:
        query_embedding (np.ndarray): Embedding da consulta.
        index: Índice FAISS para busca.
        k (int): Número de resultados a serem retornados.

    Returns:
        np.ndarray: Índices dos pedaços mais similares.
    """
    try:
        distances, indices = index.search(query_embedding, k)
        return indices
    except Exception as e:
        print(f"Erro ao buscar no índice FAISS: {e}")
        raise

def retrieve_chunks(query: str, k: int, model, chunks: List[str], index) -> List[str]:
    """
    Recupera os k pedaços mais similares a uma consulta fornecida.

    Args:
        query (str): Texto da consulta.
        k (int): Número de pedaços a serem retornados.
        model: Modelo de embeddings utilizado.
        chunks (List[str]): Lista de pedaços de texto.
        index: Índice FAISS para busca.

    Returns:
        List[str]: Lista dos pedaços mais similares à consulta.
    """
    try:
        # Gera o embedding da consulta
        query_embedding = generate_query_embedding(query, model)
        # Realiza a busca no índice FAISS
        indices = search_index(query_embedding, index, k)
        # Recupera os pedaços correspondentes
        retrieved_chunks = [chunks[i] for i in indices[0]]
        return retrieved_chunks
    except Exception as e:
        print(f"Erro ao recuperar os pedaços: {e}")
        return []

# Exemplo de consulta para gerar o embedding
query = "Your search query here"

# Recupera os 5 documentos mais similares à consulta
similar_docs = retrieve_chunks(query, k=5, model=model_emb, chunks=chunks, index=index)

# Imprime a consulta e os pedaços recuperados
print(f"Consulta: {query}")
print("Pedaços Recuperados:")
for i, doc in enumerate(similar_docs):
    print(f"[Documento {i+1}] : {doc}")
    print("--------------------")


Consulta: Your search query here
Pedaços Recuperados:
[Documento 1] : </li><li>- <a href="https%3A//www.myspace.com/popsearch/">Official Myspace page for POPSearch 2007</a>
</li></ul>
--------------------
[Documento 2] : </li><li>- <a href="Liquid%20crystal%20tunable%20filter">Liquid crystal tunable filter</a>
</li><li>- <a href="List%20of%20Earth%20observation%20satellites">List of Earth observation satellites</a>
</li><li>- <a href="Mobile%20mapping">Mobile mapping</a>
</li><li>- <a href="Multispectral%20pattern%20recognition">Multispectral pattern recognition</a>
</li><li>- <a href="National%20Center%20for%20Remote%20Sensing%2C%20Air%20and%20Space%20Law">National Center for Remote Sensing, Air and Space Law</a>
</li><li>- <a href="National%20LIDAR%20Dataset">National LIDAR Dataset</a>
</li><li>- <a href="Orthophoto">Orthophoto</a>
</li><li>- <a href="Pictometry">Pictometry</a>
</li><li>- <a href="Radiometry">Radiometry</a>
</li><li>- <a href="Remote%20monitoring%20and%20control">Rem

In [14]:
# @title Funções

# Cria uma instância do cliente Groq, usando a chave de API fornecida
client = Groq(
    api_key=API_KEY,
)

def generate_prompt(docs: List[str], question: str) -> str:
    """
    Gera um prompt para o modelo a partir dos documentos e da pergunta.

    Args:
        docs (List[str]): Lista de documentos relevantes.
        question (str): Pergunta do usuário.

    Returns:
        str: Prompt formatado para o modelo.
    """
    # Instruções iniciais para o prompt adaptadas do segundo código
    prompt = ("Write an explanation to the answer given the documents and the question. "
              "It is important that you cite all the relevant documents. "
              "Use this format: {explanation}: 'str'\n {short answer}: 'str' \n\n")
    # Adiciona cada documento ao prompt
    for i, doc in enumerate(docs, 1):
        prompt += f"[Document {i}]: {doc}\n\n"
    # Adiciona a pergunta e solicita a resposta
    prompt += f"Question: {question}\n\nAnswer: \n\nExplanation:"
    return prompt

def extract_true_answer(hit: Dict[str, Any]) -> str:
    """
    Extrai a resposta verdadeira do item fornecido.

    Args:
        hit (Dict[str, Any]): Dicionário contendo a informação da pergunta e resposta.

    Returns:
        str: Resposta verdadeira extraída.
    """
    answer = hit.get('answer', {})
    answer_type = answer.get('type', '')
    true_answer = ""

    if answer_type == 'span':
        true_answer = ", ".join([a['text'] for a in answer.get("answer_spans", [])])
    elif answer_type == 'value':
        true_answer = f"{answer.get('answer_value', '')} {answer.get('answer_unit', '')}".strip()
    elif answer_type == 'binary':
        true_answer = answer.get('answer_value', '')
    else:
        print(f"Tipo de resposta não suportado: {answer_type}")
    return true_answer

def answer_questions(user_questions: List[Dict[str, Any]], k: int = 3) -> Dict[str, List[str]]:
    """
    Responde a uma lista de perguntas fornecidas pelo usuário.

    Args:
        user_questions (List[Dict[str, Any]]): Lista de perguntas e informações associadas.
        k (int): Número de documentos similares a serem recuperados.

    Returns:
        Dict[str, List[str]]: Dicionário com perguntas, respostas geradas e respostas verdadeiras.
    """
    results = {
        "question": [],
        "content": [],
        "answer": [],
        "explanation": []
    }

    for item in tqdm(user_questions, desc="RAG"):
        try:
            # Extrai a pergunta
            question_data = item.get('questions', [{}])[0]
            query = question_data.get('question', '')

            # Recupera os pedaços de texto mais similares à pergunta
            similar_docs = retrieve_chunks(query, k=k, model=model_emb, chunks=chunks, index=index)

            # Gera o prompt usando os documentos similares e a pergunta
            prompt = generate_prompt(similar_docs, query)

            # Responde à pergunta usando um modelo de linguagem grande (LLM)
            rag_response = client.chat.completions.create(
                messages=[
                    {
                        "role": "user",
                        "content": prompt,
                    }
                ],
                model="llama-3.2-11b-text-preview",
            )

            # Processar a resposta para separar a explicação e a resposta curta
            rag_answer = rag_response.choices[0].message.content
            rag_answer_lower = rag_answer.lower()

            # Verifica se a resposta contém "{short answer}" e "{explanation}" ou "short answer:"
            if "{short answer}" in rag_answer_lower and "{explanation}" in rag_answer_lower:
                # Separar usando os placeholders {explanation} e {short answer}
                explanation = rag_answer.split("{explanation}:")[1].split("{short answer}:")[0].strip().strip("'\"")
                short_answer = rag_answer.split("{short answer}:")[1].strip().strip("'\"")
            elif "short answer:" in rag_answer_lower:
                # Separar com "short answer:"
                split_index = rag_answer_lower.find("short answer:")
                explanation = rag_answer[:split_index].strip()
                short_answer = rag_answer[split_index + len("short answer:"):].strip()
            else:
                short_answer = rag_answer.strip()
                explanation = ''

            # Extrai a resposta verdadeira
            true_answer = extract_true_answer(question_data)

            # Armazena os resultados
            results["question"].append(query)
            results["content"].append(short_answer)
            results["answer"].append(true_answer)
            results["explanation"].append(explanation)

        except Exception as e:
            print(f"Erro ao processar a pergunta: {e}")
            continue

    return results


In [16]:
# @title Debug

import random

def display_results(results: dict):
    """
    Exibe uma amostra dos resultados obtidos após responder às perguntas.

    Args:
        results (dict): Dicionário contendo as perguntas, respostas verdadeiras e respostas geradas.
    """
    # Obtém o número de perguntas respondidas
    n = len(results.get("question", []))
    if n == 0:
        print("Nenhum resultado disponível para exibir.")
        return

    print("\n\n===> Exemplo <===")
    for i in range(n):
        # Verifica se as chaves existem no dicionário antes de acessar
        question = results["question"][i] if i < len(results["question"]) else "Pergunta não disponível"
        true_answer = results["answer"][i] if i < len(results["answer"]) else "Resposta verdadeira não disponível"
        generated_answer = results["content"][i] if i < len(results["content"]) else "Resposta gerada não disponível"

        # Imprime a pergunta, a resposta verdadeira e a resposta gerada pelo modelo
        print(f"Pergunta: {question}")
        print(f"Resposta verdadeira: {true_answer}")
        print(f"Resposta gerada: {generated_answer}")
        print("-----------------------------\n")

# Define o número de perguntas a serem selecionadas aleatoriamente
num_samples = 2

# Seleciona aleatoriamente 'num_samples' itens dos dados filtrados
random_samples = random.sample(filter_data, num_samples)

# Chama a função para responder às perguntas selecionadas aleatoriamente
results = answer_questions(random_samples)

# Exibe uma amostra dos resultados
display_results(results)


RAG: 100%|██████████| 2/2 [00:01<00:00,  1.49it/s]



===> Exemplo <===
Pergunta: Did the two Eastern Front battles spearheaded by shock troops take place in the same country?
Resposta verdadeira: no
Resposta gerada: 'No'
-----------------------------

Pergunta: How long had the Spruce Grove Mets been a team when they won the Centennial Cup?
Resposta verdadeira: 1 year
Resposta gerada: They had been a team for 11 years when they won the Centennial Cup'.
-----------------------------



In [17]:
# @title Executar tudo

def execute_all(filter_data: list, num_questions: int =150) -> dict:
    """
    Executa o processo de resposta às perguntas e retorna os resultados.

    Args:
        filter_data (list): Lista de dados filtrados contendo as perguntas.
        num_questions (int): Número de perguntas a serem respondidas.

    Returns:
        dict: Dicionário com as perguntas, respostas geradas e respostas verdadeiras.
    """
    try:
        # Chama a função para responder às perguntas, usando os primeiros 'num_questions' itens dos dados filtrados
        results = answer_questions(filter_data[:num_questions])

        # Obtém o número total de perguntas respondidas
        total_questions = len(results.get("question", []))
        print(f'Total de perguntas: {total_questions}')

        return results

    except Exception as e:
        print(f"Erro ao executar todas as perguntas: {e}")
        return {}

# Exemplo de uso da função
results = execute_all(filter_data, num_questions=150)


RAG:  37%|███▋      | 55/150 [08:59<17:11, 10.85s/it]

Erro ao processar a pergunta: list index out of range


RAG:  50%|█████     | 75/150 [12:43<13:26, 10.76s/it]

Erro ao processar a pergunta: list index out of range


RAG:  60%|██████    | 90/150 [15:09<10:21, 10.36s/it]

Erro ao processar a pergunta: list index out of range


RAG:  80%|████████  | 120/150 [20:30<05:12, 10.42s/it]

Erro ao processar a pergunta: list index out of range


RAG: 100%|██████████| 150/150 [26:10<00:00, 10.47s/it]

Total de perguntas: 146


In [18]:
# @title F1-BoW (Bag of Words)

# Função para tokenizar um texto, transformando em minúsculas e dividindo em palavras
def tokenize(text: str) -> List[str]:
    """
    Tokeniza o texto, convertendo para minúsculas e extraindo as palavras.

    Args:
        text (str): Texto a ser tokenizado.

    Returns:
        List[str]: Lista de tokens extraídos do texto.
    """
    import re
    # Utiliza expressão regular para extrair palavras, ignorando pontuação
    return re.findall(r'\b\w+\b', text.lower())

# Função para calcular a métrica F1 usando a abordagem Bag of Words
def bag_of_words_f1(reference: str, generated: str) -> float:
    """
    Calcula o F1-Score entre dois textos usando a abordagem Bag of Words.

    Args:
        reference (str): Texto de referência (resposta verdadeira).
        generated (str): Texto gerado pelo modelo.

    Returns:
        float: Valor do F1-Score entre 0 e 1.
    """
    # Tokeniza as respostas de referência e gerada
    ref_tokens = tokenize(reference)
    gen_tokens = tokenize(generated)

    # Converte para Counter (bag of words)
    ref_bag = Counter(ref_tokens)
    gen_bag = Counter(gen_tokens)

    # Caso ambos os conjuntos estejam vazios
    if not ref_bag and not gen_bag:
        return 1.0  # Considera correspondência perfeita

    # Caso um dos conjuntos esteja vazio
    if not ref_bag or not gen_bag:
        return 0.0  # Sem correspondência possível

    # Calcula a interseção dos tokens
    common_tokens = ref_bag & gen_bag
    correct = sum(common_tokens.values())

    # Se não houver tokens em comum
    if correct == 0:
        return 0.0

    # Calcula precisão e recall
    precision = correct / sum(gen_bag.values())
    recall = correct / sum(ref_bag.values())

    # Evita divisão por zero na fórmula do F1-Score
    if precision + recall == 0:
        return 0.0

    # Cálculo do F1-Score
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

# Exemplo para testar a função
reference = "This is an example reference text."
generated = "This text is an example!"

# Calcula o F1-BoW entre a resposta verdadeira e a resposta gerada
f1 = bag_of_words_f1(reference, generated)

# Imprime o exemplo e o resultado do F1-BoW
print("Exemplo de F1-BoW")
print(f"Resposta verdadeira: {reference}")
print(f"Resposta gerada pelo modelo: {generated}")
print(f"F1-Score: {f1:.4f}")


Exemplo de F1-BoW
Resposta verdadeira: This is an example reference text.
Resposta gerada pelo modelo: This text is an example!
F1-Score: 0.9091


In [19]:
# @title Avaliação de Performance


def evaluate_performance(results: dict) -> float:
    """
    Avalia o desempenho calculando o F1-Score médio para as respostas geradas.

    Args:
        results (dict): Dicionário contendo as perguntas, respostas verdadeiras e respostas geradas.

    Returns:
        float: O F1-Score médio das respostas.
    """
    try:
        # Obtém as listas de respostas verdadeiras e geradas
        true_answers = results.get("answer", [])
        generated_answers = results.get("content", [])
        num_questions = len(results.get("question", []))

        # Verifica se as listas têm o mesmo tamanho
        if not (len(true_answers) == len(generated_answers) == num_questions):
            print("Erro: O número de perguntas, respostas verdadeiras e respostas geradas não correspondem.")
            return 0.0

        if num_questions == 0:
            print("Nenhuma pergunta foi respondida.")
            return 0.0

        f1_scores = []  # Lista para armazenar o F1-Score de cada pergunta

        # Itera sobre todas as perguntas e calcula o F1-Score para cada par resposta verdadeira/resposta gerada
        for i in range(num_questions):
            true_answer = true_answers[i]
            generated_answer = generated_answers[i]
            f1 = bag_of_words_f1(true_answer, generated_answer)
            f1_scores.append(f1)

        # Calcula o F1-Score médio
        mean_f1_score = np.mean(f1_scores)

        # Imprime o número total de perguntas e o F1-Score médio
        print(f"Total de perguntas: {num_questions}")
        print(f"F1-Score médio: {mean_f1_score:.4f}")

        return mean_f1_score

    except Exception as e:
        print(f"Erro ao avaliar o desempenho: {e}")
        return 0.0

# Exemplo de uso da função
mean_f1 = evaluate_performance(results)


Total de perguntas: 146
F1-Score médio: 0.1314


In [20]:
def display_results(results: dict):
    """
    Exibe uma amostra dos resultados obtidos após responder às perguntas.

    Args:
        results (dict): Dicionário contendo as perguntas, respostas verdadeiras e respostas geradas.
    """
    # Obtém o número de perguntas respondidas
    n = len(results.get("question", []))
    if n == 0:
        print("Nenhum resultado disponível para exibir.")
        return

    print("\n\n===> Exemplo <===")
    for i in range(n):
        # Verifica se as chaves existem no dicionário antes de acessar
        question = results["question"][i] if i < len(results["question"]) else "Pergunta não disponível"
        true_answer = results["answer"][i] if i < len(results["answer"]) else "Resposta verdadeira não disponível"
        generated_answer = results["content"][i] if i < len(results["content"]) else "Resposta gerada não disponível"

        # Imprime a pergunta, a resposta verdadeira e a resposta gerada pelo modelo
        print(f"Pergunta: {question}")
        print(f"Resposta verdadeira: {true_answer}")
        print(f"Resposta gerada: {generated_answer}")
        # Adiciona um indicador se a resposta está correta ou não
        if true_answer == generated_answer:
            print("Resultado: ✅ Resposta correta")
        else:
            print("Resultado: ❌ Resposta incorreta")
        print("-----------------------------\n")

# Exibe uma amostra dos resultados
display_results(results)



===> Exemplo <===
Pergunta: What is Zeus know for in Greek mythology?
Resposta verdadeira: sky and thunder god
Resposta gerada: Bearer of the Aegis and various other roles such as king of the gods, patron of the marketplace and keeper of oaths.

According to [Document 1] and [Document 2], Zeus has many roles and epithets in Greek mythology. [Document 2] provides specific examples of his various roles, such as:

- Zeus Aegiduchos or Aegiochos, who is the bearer of the Aegis, a divine shield with Medusa's head on it.
- Zeus Agoraeus, who is the patron of the marketplace and punisher of dishonest traders.
- Zeus Areius, who is warlike or the atoning one.
- Zeus Horkios, who is the keeper of oaths and has a sanctuary at Olympia where exposed liars would dedicate a votive statue to him.
- Zeus Olympios, who is the king of the gods and patron of the Panhellenic Games at Olympia.
- Zeus Panhellenios, who is worshipped on Aeacus's temple on Aegina.
- Zeus Xenios, Philoxenon, or Hospites, who